In [27]:
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
notebook_dir = Path.cwd()
project_root = notebook_dir.parent
sys.path.append(str(project_root))


# Break Aicon 4 Ads (BAD)

In [28]:
# Import required classes
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon

## Example usage
Section 1: Create and Initialize the AIcon


In [ ]:
# Import necessary libraries
import sys
import os
import importlib
import json
from pprint import pprint

# Fix import path for Babel
project_root = "/Users/infa/Documents/Babel"
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to sys.path")

# Import and reload modules to ensure latest versions
import aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor
import aicons.definitions.simple_bad_aicon

# Reload the modules
importlib.reload(aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor)
importlib.reload(aicons.definitions.simple_bad_aicon)

# Now import the classes from freshly reloaded modules
from aicons.bayesbrainGPT.sensors.meta_s.meta_ads_sales_sensor import MetaAdsSalesSensor
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon

# Your Facebook API credentials
# You'll need to replace these with your actual credentials
access_token = "EAAZAn8wmq1IEBOZCz8oyDZBBgiazAgnQKIoAr4mFTbkV7jxi6t3APzOSxFybXNIkBgwQACdagbs5lFE8tpnNOBOOpWtS3KjZAdf9MNAlySpwEaDrX32oQwUTNmOZAaSXjT5Os5Q8YqRo57tXOUukB7QtcO8nQ8JuqrnnshCr7A0giynZBnJKfuPakrZBWoZD"  # Replace with your actual token
ad_account_id = "act_252267674525035"     # Replace with your actual ad account ID
campaign_id = "120218631288730217"           # The campaign ID you provided

# Create the Meta Ads Sensor with real API credentials
print("Creating Meta Ads Sensor with real API access...")
sensor = MetaAdsSalesSensor(
    name="meta_ads",
    reliability=0.9,
    access_token=access_token,
    ad_account_id=ad_account_id,
    campaign_id=campaign_id,
    api_version="v18.0",          # Using the latest API version
    time_granularity="hour"       # Hourly data granularity
)

# Create and initialize AIcon
aicon = SimpleBadAIcon("meta_ads_real_data")

# Add the sensor to the AIcon
aicon.add_sensor("meta_ads", sensor)

# Get initial campaign and ad set information
print("\nFetching campaign and ad set information...")
ad_sets = sensor.get_adsets_for_campaign(campaign_id)
if ad_sets:
    print(f"Found {len(ad_sets)} ad sets in campaign {campaign_id}")
    for ad_set in ad_sets:
        print(f"  - Ad Set: {ad_set['adset_name']} (ID: {ad_set['adset_id']})")
else:
    print("No ad sets found or API access failed. Check your credentials.")

# Get all ads in the campaign
print("\nFetching all ads in the campaign...")
all_ads = sensor.get_all_ads()
if all_ads:
    print(f"Found {len(all_ads)} ads across all ad sets")
    for ad in all_ads[:5]:  # Show first 5 ads to avoid clutter
        print(f"  - Ad: {ad['ad_name']} (ID: {ad['ad_id']}, Status: {ad['effective_status']})")
    if len(all_ads) > 5:
        print(f"  ... and {len(all_ads) - 5} more ads")
else:
    print("No ads found or API access failed. Check your credentials.")

# Fetch actual data from the API
print("\nFetching actual ad performance data...")
data = sensor.run()

# Show campaign level metrics
print("\nCampaign level metrics:")
print(f"  - Purchases: {data['purchases']}")
print(f"  - Add to Carts: {data['add_to_carts']}")
print(f"  - Initiated Checkouts: {data['initiated_checkouts']}")

# Show data for a few ads
print("\nSample ad-level metrics:")
ad_performances = data.get('ad_performances', {})
sample_ads = list(ad_performances.items())[:3]  # Show first 3 ads
for ad_id, ad_data in sample_ads:
    print(f"Ad {ad_id} ({ad_data.get('ad_name', 'Unknown')}):")
    print(f"  - Purchases: {ad_data.get('purchases', 0)}")
    print(f"  - Add to Carts: {ad_data.get('add_to_carts', 0)}")
    print(f"  - Impressions: {ad_data.get('impressions', 0)}")
    print(f"  - Clicks: {ad_data.get('clicks', 0)}")
    print(f"  - Spend: ${ad_data.get('spend', 0)}")

# Get expected factors to see hierarchical definitions
print("\nExamining hierarchical factor definitions:")
expected_factors = sensor.get_expected_factors()

# Display hierarchical relationships
hierarchical_factors = {name: info for name, info in expected_factors.items() 
                      if 'hierarchy' in info}

print(f"\nFound {len(hierarchical_factors)} hierarchical relationships:")
for name, info in hierarchical_factors.items():
    hierarchy = info.get('hierarchy', {})
    if hierarchy.get('role') == 'aggregate':
        print(f"  - {name} (AGGREGATE): aggregates {hierarchy.get('child_pattern')} using {hierarchy.get('aggregation')}")
    elif hierarchy.get('role') == 'child':
        print(f"  - {name} (CHILD): contributes to {hierarchy.get('parent')}")

# Update the AIcon with sensor data
print("\nUpdating AIcon with real sensor data...")
env = {"use_individual_factors": True, "avoid_sampling": True}  # Simplify for demo purposes
aicon.update_from_sensor("meta_ads", environment=env)

# Display campaign-level metrics in the state
print("\nCampaign-level metrics in state:")
state = aicon.brain.get_state_factors()
for key in ["purchases", "add_to_carts", "initiated_checkouts"]:
    if key in state:
        print(f"  - {key}: {state[key].get('value')}")
    else:
        print(f"  - {key}: Not in state")

Mock data generated with the following ads:
  - Ad ID: 1_1
  - Ad ID: 1_2
  - Ad ID: 2_1
  - Ad ID: 2_2
Auto-creating missing factor: purchases (continuous)
Creating continuous factor with TensorFlow: purchases
Added continuous factor with TensorFlow distribution: purchases
Auto-creating missing factor: add_to_carts (continuous)
Creating continuous factor with TensorFlow: add_to_carts
Added continuous factor with TensorFlow distribution: add_to_carts
Auto-creating missing factor: initiated_checkouts (continuous)
Creating continuous factor with TensorFlow: initiated_checkouts
Added continuous factor with TensorFlow distribution: initiated_checkouts
Auto-creating missing factor: ad_*_purchases (continuous)
Creating continuous factor with TensorFlow: ad_*_purchases
Added continuous factor with TensorFlow distribution: ad_*_purchases
Auto-creating missing factor: ad_*_add_to_carts (continuous)
Creating continuous factor with TensorFlow: ad_*_add_to_carts
Added continuous factor with Tensor

### Section 2: Create Action Space and Utility Function

In [21]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Import the AIcon classes after reloading
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon
from aicons.bayesbrainGPT.sensors.tf_sensors import MarketingSensor, WeatherSensor

# Create a new AIcon for marketing budget allocation
aicon = SimpleBadAIcon(name="Marketing Budget Allocator")
print(f"Created new AIcon: {aicon.name}")

Created new AIcon: Marketing Budget Allocator


### Section 3: Add Sensors (with Auto-Factor Creation)

In [22]:
# Register sensors - this will automatically create the necessary factors
marketing_sensor = MarketingSensor()
aicon.add_sensor("marketing", marketing_sensor)

weather_sensor = WeatherSensor()
aicon.add_sensor("weather", weather_sensor)

# Display all the factors that were automatically created
print("\nAIcon State after adding sensors:")
print(aicon.get_state(format_nicely=True))

Auto-creating missing factor: base_conversion_rate (continuous)
Creating continuous factor with TensorFlow: base_conversion_rate
Added continuous factor with TensorFlow distribution: base_conversion_rate
Auto-creating missing factor: primary_channel (categorical)
Creating categorical factor with TensorFlow: primary_channel
Added categorical factor with TensorFlow distribution: primary_channel
Auto-creating missing factor: optimal_daily_ads (discrete)
Creating discrete factor with TensorFlow: optimal_daily_ads
Added discrete factor with TensorFlow distribution: optimal_daily_ads
Registered TFSensor: marketing (type: MarketingSensor)
Auto-creating missing factor: temperature (continuous)
Creating continuous factor with TensorFlow: temperature
Added continuous factor with TensorFlow distribution: temperature
Auto-creating missing factor: weather_condition (categorical)
Creating categorical factor with TensorFlow: weather_condition
Added categorical factor with TensorFlow distribution: wea

### Section 4: Create the Action Space


In [23]:
# Define the action space for our decision problem
aicon.create_action_space(
    space_type='marketing',
    total_budget=1000.0,
    num_ads=3,
    budget_step=10.0,
    min_budget=50.0,
    ad_names=['search', 'display', 'social']
)

print("\nAction space created")
print(f"Number of dimensions: {len(aicon.action_space.dimensions)}")
print(f"Dimensions: {[dim.name for dim in aicon.action_space.dimensions]}")

Created marketing action space with 3 dimensions

Action space created
Number of dimensions: 3
Dimensions: ['search_budget', 'display_budget', 'social_budget']


### Section 5: Create the Utility Function

In [24]:
# Define our utility function (what we're trying to maximize)
aicon.create_utility_function(
    utility_type='weather_dependent_marketing_roi',
    revenue_per_sale=15.0,
    
    # Define how weather affects different ad channels
    weather_effects={
        'sunny': {'search': 1.0, 'display': 0.8, 'social': 1.2},
        'rainy': {'search': 1.1, 'display': 1.2, 'social': 0.9},
        'cloudy': {'search': 1.2, 'display': 1.0, 'social': 0.8},
        'snowy': {'search': 0.9, 'display': 0.7, 'social': 1.1}
    }
)

print("\nUtility function created")
print(f"Type: {aicon.utility_function.name}")
print(f"Description: {aicon.utility_function.description}")

Created weather_dependent_marketing_roi utility function: Marketing ROI Utility
Description: Calculates ROI with weather condition adjustments

Utility function created
Type: Marketing ROI Utility
Description: Calculates ROI with weather condition adjustments


### Section 6: Run Perception to Update Beliefs


In [25]:
# Create simulated data for our sensors
simulated_data = {
    # Marketing data
    "base_conversion_rate": (0.05, 0.9),
    "primary_channel": ("social", 0.8),
    "optimal_daily_ads": (12, 0.7),
    
    # Weather data
    "temperature": (23.5, 0.95),
    "weather_condition": ("sunny", 0.9),
    "precipitation": (0.0, 0.95)
}

# Run a single perception update to incorporate the data
print("\nRunning perception update...")
aicon.run(mode='once', environment=simulated_data)

# Check if we have posterior samples
posterior_samples = aicon.get_posterior_samples()
if posterior_samples:
    print("\nPosterior samples available for factors:")
    for factor_name in posterior_samples:
        samples = posterior_samples[factor_name]
        if isinstance(samples, np.ndarray) and samples.size > 0:
            print(f"  {factor_name}: {len(samples)} samples, mean={np.mean(samples):.4f}")
else:
    print("\nNo posterior samples available. Check for errors in the perception update.")


Running perception update...
Running single perception update...


TypeError: can only concatenate tuple (not "float") to tuple

In [26]:
# Reload modules to pick up our debug statements
import importlib
import aicons.definitions.simple_bad_aicon
import aicons.bayesbrainGPT.sensors.tf_sensors
import aicons.bayesbrainGPT.perception.perception

importlib.reload(aicons.definitions.simple_bad_aicon)
importlib.reload(aicons.bayesbrainGPT.sensors.tf_sensors)
importlib.reload(aicons.bayesbrainGPT.perception.perception)
print("Modules reloaded successfully")

# Setup
from aicons.definitions.simple_bad_aicon import SimpleBadAIcon
from aicons.bayesbrainGPT.sensors.tf_sensors import MarketingSensor

# Create AIcon
aicon = SimpleBadAIcon(name="Marketing Budget Allocator")

# Add sensors
marketing_sensor = MarketingSensor()
aicon.add_sensor("marketing", marketing_sensor)

# Create simulated data - just raw values
simulated_data = {
    "base_conversion_rate": 0.05,  # Using raw value (not a tuple)
    "primary_channel": "social",
    "optimal_daily_ads": 12
}

# Run with debug tracing active
print("\nRunning perception update...")
aicon.run(mode='once', environment=simulated_data)

Modules reloaded successfully
Auto-creating missing factor: base_conversion_rate (continuous)
Creating continuous factor with TensorFlow: base_conversion_rate
Added continuous factor with TensorFlow distribution: base_conversion_rate
Auto-creating missing factor: primary_channel (categorical)
Creating categorical factor with TensorFlow: primary_channel
Added categorical factor with TensorFlow distribution: primary_channel
Auto-creating missing factor: optimal_daily_ads (discrete)
Creating discrete factor with TensorFlow: optimal_daily_ads
Added discrete factor with TensorFlow distribution: optimal_daily_ads
Registered TFSensor: marketing (type: MarketingSensor)

Running perception update...
Running single perception update...
DEBUG - Inside collect_sensor_data
DEBUG - Environment type: <class 'dict'>
DEBUG - Environment keys: ['base_conversion_rate', 'primary_channel', 'optimal_daily_ads']
DEBUG - base_conversion_rate value: 0.05
DEBUG - base_conversion_rate type: <class 'float'>
DEBUG

2025-03-13 12:58:14.809553: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at sparse_xent_op.cc:103 : INVALID_ARGUMENT: Received a label value of 4 which is outside the valid range of [0, 4).  Label values: 4
2025-03-13 12:58:14.809569: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Received a label value of 4 which is outside the valid range of [0, 4).  Label values: 4
	 [[{{function_node mcmc_sample_chain_trace_scan_while_smart_for_loop_while_simple_step_size_adaptation___init____one_step_mh_one_step_hmc_kernel_one_step_leapfrog_integrate_while_body_2688}}{{node mcmc_sample_chain/trace_scan/while/smart_for_loop/while/simple_step_size_adaptation___init__/_one_step/mh_one_step/hmc_kernel_one_step/leapfrog_integrate/while/leapfrog_integrate_one_step/maybe_call_fn_and_grads/value_and_gradients/value_and_gradient/JointDistributionNamed_CONSTRUCTED_AT_top_level/log_prob/Categorical/log_prob/SparseSoftmaxCr

{'iterations': 1,
 'start_time': datetime.datetime(2025, 3, 13, 12, 58, 13, 804370),
 'last_update_time': datetime.datetime(2025, 3, 13, 12, 58, 14, 842360),
 'updates': [{'time': datetime.datetime(2025, 3, 13, 12, 58, 14, 842364),
   'success': False,
   'sensor': 'all',
   'duration_sec': 1.0378880500793457}]}